# RSNA-2025 統合ワークフロー (exp0001_baseline)

このノートブックは training → evaluation → inference を1本で実行します。

- セッション内で完結（モデルはメモリ/ローカルに保存）
- 実行順: 上から順に


---

### 由来ノートブック: training.ipynb

# RSNA-2025 ベースライン学習 (exp0001_baseline)

このノートブックでは、ベースラインモデル（GradientBoosting）の学習を実行します。

- 実験ID: exp0001_baseline
- モデル: GradientBoostingClassifier
- 特徴量: 年齢、性別、モダリティ
- 目的変数: Aneurysm Present

## 実験設定

実験の再現性を確保するため、以下の設定を使用します：
- SEED = 130
- test_size = 0.2
- stratified split


In [ ]:
# 0) セットアップ（Colab）
import os
import sys
import subprocess
from pathlib import Path

IN_COLAB = 'google.colab' in sys.modules
print('IN_COLAB =', IN_COLAB)

if IN_COLAB:
    # pip を Python から実行
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-U', 'pip'], check=True)
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q',
                    'pandas', 'polars', 'seaborn', 'scikit-learn', 'matplotlib', 'gcsfs', 'fsspec'], check=True)

    # GCP 認証（ADC）
    from google.colab import auth  # type: ignore
    auth.authenticate_user()

    # 作業ディレクトリを設定
    os.chdir('/content')
    
    # GitHub から本リポジトリを取得
    REPO_URL = 'https://github.com/Kohei-Arita/RSNA-2025.git'
    REPO_DIR = Path('/content/RSNA-2025')
    if not REPO_DIR.exists():
        subprocess.run(['git', 'clone', REPO_URL], check=True)
    os.chdir('/content/RSNA-2025')

    # リポジトリの src を追加
    sys.path.insert(0, str(Path.cwd() / 'src'))

# GCS バケット設定
GCS_BUCKET = 'rsna2025-prod'
GCS_BASE = f'gs://{GCS_BUCKET}'
print('GCS_BASE =', GCS_BASE)


In [ ]:
# 1) データ読込
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier

SEED = 130

train_uri = f'{GCS_BASE}/train.csv'

# ColabのADCを gcsfs が利用
train = pd.read_csv(train_uri, storage_options={'token': 'cloud'})

print(f"Number of training series: {train.shape[0]}")
display(train.head())


In [ ]:
# 2) 特徴量エンジニアリング
# 年齢を数値化（"xx - yy" 形式の先頭、または数字抽出）
df_age_str = train['PatientAge'].astype(str)
age_first = df_age_str.str.split(' - ').str[0]
age_vals = pd.to_numeric(age_first.str.extract(r'([0-9]+(?:\.[0-9]+)?)')[0], errors='coerce')

# 特徴量作成: 年齢・性別（Male=1）・モダリティone-hot
x_age = age_vals.fillna(age_vals.median())
X = pd.DataFrame({
    'age': x_age,
    'sex': (train['PatientSex'] == 'Male').astype(int)
})
mod_dummies = pd.get_dummies(train['Modality'], prefix='mod')
X = pd.concat([X, mod_dummies], axis=1)

# 目的変数
if train['Aneurysm Present'].dtype != np.int64 and train['Aneurysm Present'].dtype != np.int32:
    y = train['Aneurysm Present'].astype(int)
else:
    y = train['Aneurysm Present']

print(f"Feature matrix shape: {X.shape}")
print(f"Target distribution: {y.value_counts()}")


In [ ]:
# 3) 学習・検証データ分割
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED
)

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Training target distribution:\n{y_train.value_counts()}")
print(f"Validation target distribution:\n{y_val.value_counts()}")


In [ ]:
# 4) モデル学習
gbm = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=8,
    random_state=SEED
)

gbm.fit(X_train, y_train)

# 訓練データでの予測
train_probs = gbm.predict_proba(X_train)[:, 1]
train_auc = roc_auc_score(y_train, train_probs)
print(f"GBM Training AUC: {train_auc:.4f}")

# 検証データでの予測
val_probs = gbm.predict_proba(X_val)[:, 1]
val_auc = roc_auc_score(y_val, val_probs)
print(f"GBM Validation AUC: {val_auc:.4f}")


In [ ]:
# 5) 特徴量重要度の可視化
import matplotlib.pyplot as plt
import seaborn as sns

feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': gbm.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance.head(10), x='importance', y='feature')
plt.title('Top 10 Feature Importances')
plt.xlabel('Importance')
plt.show()

display(feature_importance)


In [ ]:
# 6) モデルと設定の保存
import pickle
import json

# モデルの保存
models_dir = Path('models/exp0001_baseline')
models_dir.mkdir(parents=True, exist_ok=True)

with open(models_dir / 'gbm_baseline.pkl', 'wb') as f:
    pickle.dump(gbm, f)

# 列名の保存（推論で必要）
MOD_COLUMNS = list(mod_dummies.columns)
metadata = {
    'feature_columns': list(X.columns),
    'mod_columns': MOD_COLUMNS,
    'train_auc': float(train_auc),
    'val_auc': float(val_auc),
    'seed': SEED,
    'model_params': gbm.get_params()
}

with open(models_dir / 'metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"Model saved to: {models_dir}")
print(f"Training AUC: {train_auc:.4f}")
print(f"Validation AUC: {val_auc:.4f}")


---

### 由来ノートブック: evaluation.ipynb

# RSNA-2025 ベースライン評価 (exp0001_baseline)

このノートブックでは、学習済みモデルの評価とOut-of-Fold予測を実行します。

- 実験ID: exp0001_baseline
- モデル: 学習済みGradientBoostingClassifier
- 評価指標: AUC、閾値最適化、混同行列

## 主な内容
1. 学習済みモデルのロード
2. Out-of-Fold（OOF）予測
3. 閾値最適化
4. 評価指標の可視化


In [ ]:
# 0) セットアップ
import os
import sys
import subprocess
from pathlib import Path
import pickle
import json

IN_COLAB = 'google.colab' in sys.modules
print('IN_COLAB =', IN_COLAB)

if IN_COLAB:
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-U', 'pip'], check=True)
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q',
                    'pandas', 'polars', 'seaborn', 'scikit-learn', 'matplotlib', 'gcsfs', 'fsspec'], check=True)
    from google.colab import auth
    auth.authenticate_user()
    os.chdir('/content')
    REPO_URL = 'https://github.com/Kohei-Arita/RSNA-2025.git'
    REPO_DIR = Path('/content/RSNA-2025')
    if not REPO_DIR.exists():
        subprocess.run(['git', 'clone', REPO_URL], check=True)
    os.chdir('/content/RSNA-2025')
    sys.path.insert(0, str(Path.cwd() / 'src'))

GCS_BUCKET = 'rsna2025-prod'
GCS_BASE = f'gs://{GCS_BUCKET}'
print('GCS_BASE =', GCS_BASE)


In [ ]:
# 1) モデルとメタデータのロード
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, classification_report

models_dir = Path('models/exp0001_baseline')

# モデルのロード
with open(models_dir / 'gbm_baseline.pkl', 'rb') as f:
    gbm = pickle.load(f)

# メタデータのロード
with open(models_dir / 'metadata.json', 'r') as f:
    metadata = json.load(f)

print("Model loaded successfully")
print(f"Training AUC: {metadata['train_auc']:.4f}")
print(f"Validation AUC: {metadata['val_auc']:.4f}")
print(f"Feature columns: {metadata['feature_columns'][:5]}...")


In [ ]:
# 2) データの再読み込みと特徴量作成
train_uri = f'{GCS_BASE}/train.csv'
train = pd.read_csv(train_uri, storage_options={'token': 'cloud'})

# 特徴量の再作成（training.ipynbと同じ処理）
df_age_str = train['PatientAge'].astype(str)
age_first = df_age_str.str.split(' - ').str[0]
age_vals = pd.to_numeric(age_first.str.extract(r'([0-9]+(?:\.[0-9]+)?)')[0], errors='coerce')

x_age = age_vals.fillna(age_vals.median())
X = pd.DataFrame({
    'age': x_age,
    'sex': (train['PatientSex'] == 'Male').astype(int)
})
mod_dummies = pd.get_dummies(train['Modality'], prefix='mod')
X = pd.concat([X, mod_dummies], axis=1)

# 目的変数
if train['Aneurysm Present'].dtype != np.int64 and train['Aneurysm Present'].dtype != np.int32:
    y = train['Aneurysm Present'].astype(int)
else:
    y = train['Aneurysm Present']

print(f"Data loaded: {X.shape}")


In [ ]:
# 3) Out-of-Fold (OOF) 予測
from sklearn.model_selection import StratifiedKFold

SEED = metadata['seed']
n_splits = 5

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
oof_preds = np.zeros(len(X))

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    X_train_fold = X.iloc[train_idx]
    y_train_fold = y.iloc[train_idx]
    X_val_fold = X.iloc[val_idx]
    
    # フォールドごとにモデルを学習
    from sklearn.ensemble import GradientBoostingClassifier
    fold_model = GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=8,
        random_state=SEED
    )
    fold_model.fit(X_train_fold, y_train_fold)
    
    # OOF予測
    oof_preds[val_idx] = fold_model.predict_proba(X_val_fold)[:, 1]
    
    # フォールドごとのAUC
    fold_auc = roc_auc_score(y.iloc[val_idx], oof_preds[val_idx])
    print(f"Fold {fold+1} AUC: {fold_auc:.4f}")

# 全体のOOF AUC
overall_oof_auc = roc_auc_score(y, oof_preds)
print(f"\nOverall OOF AUC: {overall_oof_auc:.4f}")


In [ ]:
# 4) ROC曲線と閾値最適化
import matplotlib.pyplot as plt
import seaborn as sns

fpr, tpr, thresholds = roc_curve(y, oof_preds)

# Youden's J統計量で最適閾値を探索
youden_j = tpr - fpr
optimal_idx = np.argmax(youden_j)
optimal_threshold = thresholds[optimal_idx]

plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {overall_oof_auc:.4f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.scatter(fpr[optimal_idx], tpr[optimal_idx], color='red', s=100, 
           label=f'Optimal threshold = {optimal_threshold:.3f}')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Aneurysm Present')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Optimal Threshold: {optimal_threshold:.3f}")
print(f"Sensitivity at optimal: {tpr[optimal_idx]:.3f}")
print(f"Specificity at optimal: {1-fpr[optimal_idx]:.3f}")


In [ ]:
# 5) 混同行列と分類レポート
from sklearn.metrics import ConfusionMatrixDisplay

y_pred_binary = (oof_preds >= optimal_threshold).astype(int)

# 混同行列の表示
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 正規化なし
cm = confusion_matrix(y, y_pred_binary)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No Aneurysm', 'Aneurysm'])
disp.plot(ax=axes[0], cmap='Blues')
axes[0].set_title('Confusion Matrix (Count)')

# 正規化あり
cm_norm = confusion_matrix(y, y_pred_binary, normalize='true')
disp_norm = ConfusionMatrixDisplay(confusion_matrix=cm_norm, display_labels=['No Aneurysm', 'Aneurysm'])
disp_norm.plot(ax=axes[1], cmap='Blues')
axes[1].set_title('Confusion Matrix (Normalized)')

plt.tight_layout()
plt.show()

# 分類レポート
print("\nClassification Report:")
print(classification_report(y, y_pred_binary, target_names=['No Aneurysm', 'Aneurysm']))


In [ ]:
# 6) OOF予測の保存
outputs_dir = Path('outputs/oof/exp0001_baseline')
outputs_dir.mkdir(parents=True, exist_ok=True)

# OOF予測をDataFrameとして保存
oof_df = pd.DataFrame({
    'SeriesInstanceUID': train['SeriesInstanceUID'],
    'Aneurysm Present': y,
    'oof_pred': oof_preds
})

oof_df.to_csv(outputs_dir / 'oof_predictions.csv', index=False)

# 評価メトリクスの保存
metrics = {
    'overall_oof_auc': float(overall_oof_auc),
    'optimal_threshold': float(optimal_threshold),
    'sensitivity': float(tpr[optimal_idx]),
    'specificity': float(1-fpr[optimal_idx])
}

with open(outputs_dir / 'metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"OOF predictions saved to: {outputs_dir}")
print(f"Overall OOF AUC: {overall_oof_auc:.4f}")


---

### 由来ノートブック: inference.ipynb

# RSNA-2025 ベースライン推論 (exp0001_baseline)

このノートブックでは、学習済みモデルを使用した推論と提出ファイル生成（参考用）を行います。

- 実験ID: exp0001_baseline
- モデル: 学習済みGradientBoostingClassifier
- 出力: 14ラベルの予測確率

## 主な内容
1. 学習済みモデルのロード
2. 特徴量作成関数の定義
3. 予測関数の実装
4. サンプル予測の実行


In [ ]:
# 0) セットアップ
import os
import sys
import subprocess
from pathlib import Path
import pickle
import json

IN_COLAB = 'google.colab' in sys.modules
print('IN_COLAB =', IN_COLAB)

if IN_COLAB:
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-U', 'pip'], check=True)
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q',
                    'pandas', 'polars', 'seaborn', 'scikit-learn', 'matplotlib', 'gcsfs', 'fsspec'], check=True)
    from google.colab import auth
    auth.authenticate_user()
    os.chdir('/content')
    REPO_URL = 'https://github.com/Kohei-Arita/RSNA-2025.git'
    REPO_DIR = Path('/content/RSNA-2025')
    if not REPO_DIR.exists():
        subprocess.run(['git', 'clone', REPO_URL], check=True)
    os.chdir('/content/RSNA-2025')
    sys.path.insert(0, str(Path.cwd() / 'src'))

GCS_BUCKET = 'rsna2025-prod'
GCS_BASE = f'gs://{GCS_BUCKET}'
print('GCS_BASE =', GCS_BASE)


In [ ]:
# 1) モデルとメタデータのロード
import pandas as pd
import polars as pl
import numpy as np

models_dir = Path('models/exp0001_baseline')

# モデルのロード
with open(models_dir / 'gbm_baseline.pkl', 'rb') as f:
    gbm = pickle.load(f)

# メタデータのロード（特徴量の列名など）
with open(models_dir / 'metadata.json', 'r') as f:
    metadata = json.load(f)

MOD_COLUMNS = metadata['mod_columns']
print("Model loaded successfully")
print(f"Feature columns: {metadata['feature_columns'][:5]}...")
print(f"Modality columns: {MOD_COLUMNS}")


In [ ]:
# 2) ラベル定義（提出用）
ID_COL = 'SeriesInstanceUID'
LABEL_COLS = [
    'Left Infraclinoid Internal Carotid Artery',
    'Right Infraclinoid Internal Carotid Artery',
    'Left Supraclinoid Internal Carotid Artery',
    'Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery',
    'Right Middle Cerebral Artery',
    'Anterior Communicating Artery',
    'Left Anterior Cerebral Artery',
    'Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery',
    'Right Posterior Communicating Artery',
    'Basilar Tip',
    'Other Posterior Circulation',
    'Aneurysm Present'
]

print(f"Number of labels: {len(LABEL_COLS)}")


In [ ]:
# 3) 訓練データの読み込み（年齢の中央値計算用）
train_uri = f'{GCS_BASE}/train.csv'
train = pd.read_csv(train_uri, storage_options={'token': 'cloud'})

# 年齢の中央値を計算（欠損値埋め用）
df_age_str = train['PatientAge'].astype(str)
age_first = df_age_str.str.split(' - ').str[0]
age_vals = pd.to_numeric(age_first.str.extract(r'([0-9]+(?:\.[0-9]+)?)')[0], errors='coerce')
AGE_MEDIAN = age_vals.median()
print(f"Age median for imputation: {AGE_MEDIAN:.1f}")


In [ ]:
# 4) 特徴量作成関数
def build_feature_row(row):
    """単一行から特徴量を作成"""
    # 年齢の処理
    age_str = str(row.get('PatientAge', ''))
    age_val = pd.to_numeric(age_str.split(' - ')[0], errors='coerce')
    if pd.isna(age_val):
        age_val = AGE_MEDIAN
    
    # 性別の処理
    sex_val = 1 if row.get('PatientSex', '') == 'Male' else 0
    
    # 特徴量辞書を作成
    feats = {'age': age_val, 'sex': sex_val}
    
    # モダリティのone-hot encoding
    modality = row.get('Modality', '')
    for m in MOD_COLUMNS:
        feats[m] = 1 if m == f"mod_{modality}" else 0
    
    return pd.DataFrame([feats])


In [ ]:
# 5) 予測関数
def predict_series(series_data):
    """シリーズデータから14ラベルの予測を生成"""
    # 特徴量を作成
    feat_df = build_feature_row(series_data)
    
    # presence (Aneurysm Present) の予測
    presence_prob = float(gbm.predict_proba(feat_df)[:, 1][0])
    
    # 簡略化：全ラベルに同じ確率を使用
    # 実際の提出では、ラベルごとに専用モデルを使用すべき
    predictions = {label: presence_prob for label in LABEL_COLS}
    
    return predictions


In [ ]:
# 6) サンプル予測（学習データの先頭N件で動作確認）
def build_submission_preview(n=10):
    """学習データの先頭N件で予測を生成（動作確認用）"""
    rows = []
    
    for _, row in train.head(n).iterrows():
        series_id = row[ID_COL]
        predictions = predict_series(row.to_dict())
        
        # 行を作成（SeriesInstanceUID + 14ラベルの予測）
        row_data = [series_id] + [predictions[label] for label in LABEL_COLS]
        rows.append(row_data)
    
    # Polars DataFrameとして返す
    df = pl.DataFrame(rows, schema=[ID_COL] + LABEL_COLS)
    return df

# 動作確認
submission_preview = build_submission_preview(10)
print("Submission preview:")
display(submission_preview.head())


In [ ]:
# 7) 予測結果の保存（サンプル）
outputs_dir = Path('outputs/preds/exp0001_baseline')
outputs_dir.mkdir(parents=True, exist_ok=True)

# CSVとして保存（参考用）
submission_preview.write_csv(outputs_dir / 'sample_predictions.csv')

print(f"Sample predictions saved to: {outputs_dir}")
print(f"Shape: {submission_preview.shape}")
print("\nNote: 本番のKaggle提出では、サービングAPIを使用します。")
print("このCSVは動作確認・ローカルテスト用です。")
